In [99]:
import numpy as np
import pandas as pd

In [103]:
dat = pd.read_pickle("../data/elmondo_es.pkl")

In [104]:
dat.head()

,headline,keyfacts,content,tags,time
http://www.elmundo.es/america/2014/01/02/52c4d39a22601d6f658b457c.html,Una jueza del Tribunal Supremo suspende parcia...,[ Decidió atender a los grupos conservadores y...,"El martes, antes de presidir la fiesta de fin ...",[],2014-01-02
http://www.elmundo.es/america/2014/01/02/52c4d99622601d6d658b458a.html,'La revolución cubana sigue sin compromisos co...,[ 'Jamás hemos cedido ni cederemos ante agresi...,El presidente Raúl Castro reveló que se está i...,[],2014-01-02
http://www.elmundo.es/america/2014/01/03/52c61ede268e3e3c528b456b.html,La NSA trabaja en un ordenador cuántico capaz ...,[ La información proviene de los documentos de...,La Agencia de Seguridad Nacional (NSA) trabaja...,[],2014-01-03
http://www.elmundo.es/america/2014/01/10/52cfbb62ca47415a218b456b.html,Último adiós a la ex Miss Venezuela Mónica Spe...,[ Mónica Spear y su marido fueron asesinados e...,Esta semana Venezuela ha recibido una noticia ...,[],2014-01-10
http://www.elmundo.es/america/2014/01/14/52d4b8ba268e3eb2318b456a.html,Michoacán pone en jaque al Gobierno de Peña Nieto,[ El Gobierno envía más policías y militares y...,La situación en el Estado mexicano de Michoacá...,[],2014-01-14


In [122]:
N = 3
dat['keyfacts'].iloc[N]

[' Mónica Spear y su marido fueron asesinados en presencia de la hija de ambos de 5 años ',
 ' Han detenido a 7 personas intregrantes de la banda "Los Sanguinarios de El Cambur" ',
 ' Venezuela tiene una gran tasa de criminalidad: 39 asesinatos por cada 100.000 habitantes ']

In [123]:
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

# Instantiates a client
client = language.LanguageServiceClient()

# The text to analyze
text = "\n".join(dat['keyfacts'].iloc[N]) + '\n\n' + dat['content'].iloc[N]

document = types.Document(
    content=text,
    type=enums.Document.Type.PLAIN_TEXT)

# Detects the sentiment of the text
response = client.analyze_entities(document=document)

print('Text: \n{}'.format(text))

Text: 
 Mónica Spear y su marido fueron asesinados en presencia de la hija de ambos de 5 años 
 Han detenido a 7 personas intregrantes de la banda "Los Sanguinarios de El Cambur" 
 Venezuela tiene una gran tasa de criminalidad: 39 asesinatos por cada 100.000 habitantes 

Esta semana Venezuela ha recibido una noticia que ha causado gran consternación en todo el país: el asesinato de la actriz, modelo y ex Miss Venezuela Mónica Spear y de su esposo, el irlandés Thomas Henry Berry . 
El fatídico hecho se produjo en una autopista Venezolana cuando el matrimonio se disponía a regresar a su casa. En el interior del vehículo, se hallaron los cuerpos sin vida de la joven pareja y a la hija de ambos, de 5 años que también resultó herida.
Miles de personas acudieron al velatorio de la pareja. Tanto artistas como familiares y allegados de la actriz y el empresario europeo acudieron desde el mediodía al Cementerio del Este de Caracas para dar el último adiós a la pareja.
Seguidores de la exreina d

In [124]:
# entity types from enums.Entity.Type
entity_type = ('UNKNOWN', 'PERSON', 'LOCATION', 'ORGANIZATION',
               'EVENT', 'WORK_OF_ART', 'CONSUMER_GOOD', 'OTHER')

mention_type = ('TYPE_UNKNOWN', 'PROPER', 'COMMON')

for entity in response.entities:
    print('=' * 20)
    print(u'{:<16}: {}'.format('name', entity.name))
    print(u'{:<16}: {}'.format('type', entity_type[entity.type]))
#     print(u'{:<16}: {}'.format('metadata', entity.metadata))
    print(u'{:<16}: {}'.format('salience', entity.salience))
    print(u'{:<16}: {}'.format('wikipedia_url',
          entity.metadata.get('wikipedia_url', '-')))

name            : venezolano
type            : LOCATION
salience        : 0.2702484726905823
wikipedia_url   : https://en.wikipedia.org/wiki/Venezuela
name            : amiga
type            : PERSON
salience        : 0.11774305999279022
wikipedia_url   : -
name            : noticia
type            : OTHER
salience        : 0.09770665317773819
wikipedia_url   : -
name            : empresario
type            : PERSON
salience        : 0.09743772447109222
wikipedia_url   : -
name            : Mónica Spear
type            : PERSON
salience        : 0.05869424343109131
wikipedia_url   : https://en.wikipedia.org/wiki/M%C3%B3nica_Spear
name            : ambos
type            : PERSON
salience        : 0.04923838749527931
wikipedia_url   : -
name            : ejemplo
type            : OTHER
salience        : 0.02078082598745823
wikipedia_url   : -
name            : tasa
type            : OTHER
salience        : 0.015775762498378754
wikipedia_url   : -
name            : pueblo
type            

In [125]:
def has_proper_mentions(mentions):
    return np.any([mention.type == 1 for mention in mentions])

In [126]:
def process_response(response):
    entities = response.entities
    return [{'entity_name': ent.name,
             'entity_type': entity_type[ent.type], 
            'mentions': set(mention.text.content
                        for mention in ent.mentions if mention.type == 1)} 
            for ent in entities if has_proper_mentions(ent.mentions)]

In [127]:
pd.DataFrame(process_response(response))

,entity_name,entity_type,mentions
0,venezolano,LOCATION,"{Venezuela, venezolano}"
1,Mónica Spear,PERSON,"{Mónica, Spear, Mónica Spear}"
2,Miss Venezuela,PERSON,{Miss Venezuela}
3,Los Sanguinarios de El Cambur,WORK_OF_ART,{Los Sanguinarios de El Cambur}
4,irlandés,PERSON,{irlandés}
5,Thomas Henry Berry,PERSON,{Thomas Henry Berry}
6,Rafael Spear,PERSON,{Rafael Spear}
7,europeo,LOCATION,{europeo}
8,Cementerio del Este de Caracas,LOCATION,{Cementerio del Este de Caracas}
9,Henry,PERSON,{Henry}
